# Snow Cover Interactive Chart

## Table of Contents

#### [Plot Snow Cover Data](#bullet1)
* [Packages](#package1)
* [Process Snow Data](#bullet2)
* [Calculate Snow Cover Extent](#bullet3)
* [Calculate Monthly Snow Cover Anomolies](#bullet5)
* [Select Individual Snow Cover Years and Months](#bullet6)

#### [Plot Albedo Data](#bullet7)

* [Calculate Monthly Albedo Averages](#bullet8)
* [Calculate Annual Albedo Percentages](#bullet9)
* [Calculate Monthly Albedo Anomalies](#bullet10)
* [Select Individual Years and Months for Albedo](#bullet11)

#### [Create Interactive Visuals]


# Plot Snow Cover Data<a class="anchor" id="bullet1"></a>

### Packages<a class="anchor" id="package1"></a> 

In [1]:
import glob
import os
from osgeo import gdal
import numpy as np
import pandas as pd
import plotly.graph_objects as go

### Glob the Datasets

In [10]:
# Glob will string together a list that matches the unique keys provided.
# In this case, we're creating a string of all of our snow cover datasets. 
snow_ds = glob.glob('snow_cover/*.h5')
snow_ds

['snow_cover\\Sierra2001.h5',
 'snow_cover\\Sierra2002.h5',
 'snow_cover\\Sierra2003.h5',
 'snow_cover\\Sierra2004.h5',
 'snow_cover\\Sierra2005.h5',
 'snow_cover\\Sierra2006.h5',
 'snow_cover\\Sierra2007.h5',
 'snow_cover\\Sierra2008.h5',
 'snow_cover\\Sierra2009.h5',
 'snow_cover\\Sierra2010.h5',
 'snow_cover\\Sierra2011.h5',
 'snow_cover\\Sierra2012.h5',
 'snow_cover\\Sierra2013.h5',
 'snow_cover\\Sierra2014.h5',
 'snow_cover\\Sierra2015.h5',
 'snow_cover\\Sierra2016.h5',
 'snow_cover\\Sierra2017.h5',
 'snow_cover\\Sierra2018.h5',
 'snow_cover\\Sierra2019.h5']

## Process Snow Data<a class="anchor" id="bullet2"></a>

In [12]:
# Get sub datasets of first snow fraction dataset.
datasets = gdal.Open(snow_ds[0], gdal.GA_ReadOnly).GetSubDatasets()

# (datasets[3] is to choose the 4th dataset in the subdirectory (i.e., snow fraction). 
# The second bracket [0] is needed to open the dataset.
snow_data = gdal.Open(datasets[3][0])

# Changes the selected dataset into an array.
snow_data_array = snow_data.ReadAsArray()

# Converts the variables to 'float' to allow us to convert the NA values (255) to 0
snow_data_float=snow_data_array.astype('float')
snow_data_float[snow_data_float == 255] = 0

# See http://drr.ikcest.org/tutorial/k8022
# Pull referencing matrix from h5 file.
ref_matrix_meta = snow_data.GetMetadata()['Grid_MODIS_GRID_500m_ReferencingMatrix'].split()
referencing_matrix = [int(ref_matrix_meta[2]), int(ref_matrix_meta[1]), int(ref_matrix_meta[0]), int(ref_matrix_meta[5]), int(ref_matrix_meta[4]), int(ref_matrix_meta[3])]

[-285750, 500, 0, 500250, 0, -500]

### 

## Calculate Snow Cover Extent<a class="anchor" id="bullet3"></a>



In [6]:
# Taking the mean average of a single day in the dataset. 
# First, divide by 100 to convert values to a percentage (now 0.0 to 1.0 instead of 0 to 100)
# Next, multiple by transform_data[1] and -transform_data[5] (since pixel size is [500m] * [-500m], but we are only interested in absolute area) to get the mean area * percentage snow fraction area covered in m^2
# Divide by 10**6 to get the area in km^2
# Divide by 1000 to make the values consistent with output of Snow Today website (They report values in thousands of km^2)   
test = snow_data_float[200,:,:]/1000 * referencing_matrix[1] * -referencing_matrix[5] / 10**6 /1000
np.sum(test)

39.62034250000001

### Calculate Snow Cover Extent Per Year

In [ ]:
# # Create an empty list to populate yearly for loop values with
# total_snow_cover_list = []

# # Choose the range of the data, which is dictated by how many files you globbed together.
# for i in range(len(snow_ds)): 

#     # Get sub datasets of first snow fraction dataset ('Sierra2001.h5').
#     datasets = gdal.Open(snow_ds[i], gdal.GA_ReadOnly).GetSubDatasets()

#     #(datasets[3] is to choose the 4th dataset in the subdirectory (i.e., snow fraction). 
#     #The second bracket [0] is needed to open the dataset.
#     snow_data = gdal.Open(datasets[3][0])

#     #Changes the selected dataset into an array.
#     snow_data_array = snow_data.ReadAsArray()

#     #Converts the variables to 'float' to allow us to convert the NA values (255) to 0
#     snow_data_float=snow_data_array.astype('float')
#     snow_data_float[snow_data_float == 255] = 0

#     #Create an empty list to populate daily mean values per year with
#     new_list = []
    
#     #Takes the sum of the average snow fraction area per cell per given day in a year
#     for j in range(len(snow_data_float)):
        
#         total_snow_cover = snow_data_float[j,:,:] /100 * referencing_matrix[1] * -referencing_matrix[5] / 10**6 /1000
#         total_snow_cover_year = np.sum(total_snow_cover)

#         #Appends daily values to list
#         new_list.append(total_snow_cover_year)

#     #Appends yearly values to list. 
#     total_snow_cover_list.append(new_list)

### Convert For Loop to DataFrame/Add "Names" for Rows and Columns

In [8]:
# # Create empty lists for row names and col names
# row_names = []
# col_names = []

# # Create names for each day
# for i in range(366) :

#     col_names.append(str(i + 1))

# # Create names for year 
# for j in range(len(snow_ds)):

#     row_names.append(str(j + 2001))

# #Create dataframe and append with names. 
# snow_cover_df = pd.DataFrame(total_snow_cover_list, columns= col_names, index = row_names)

# # Transpose dataframe so years are columns and days are rows.
# snow_cover_df = pd.DataFrame.transpose(snow_cover_df)

### Adding Statistical Analysis to DataFrame

In [ ]:
#snow_cover_df.to_csv('snow_cover_df.csv')
snow_cover_df = pd.read_csv(r'snow_cover_df.csv')

#Lets see what the dataset looks like
snow_cover_df

In [10]:
#Now need to convert NA Values for non-leap years
snow_extent_df = snow_cover_df.fillna(0)

### Calculate Interquartile Ranges 

In [11]:
# Create empty list to input with for loop
IQR_25 = []
IQR_75 = []
IQR_50 = []
days = []
for i in range(len(snow_extent_df)): 
    #Takes the IQR of each day (25/75)
    Q1 = np.percentile(snow_extent_df.iloc[i], 25)
    Q2 = np.percentile(snow_extent_df.iloc[i], 50)
    Q3 = np.percentile(snow_extent_df.iloc[i], 75)
    #appends list with IQR outputs
    IQR_25.append(Q1)
    IQR_50.append(Q2)
    IQR_75.append(Q3)
    #Creates day list to append dataset with
    days.append(i + 1)
    
# Next, need to create a single column of mean values. 
snow_extent_df['Average Snow Cover'] = snow_extent_df.mean(axis = 1)

In [12]:
#Appends list for loop lists
snow_extent_df['IQR_25'] = IQR_25
snow_extent_df['IQR_75'] = IQR_75
snow_extent_df['IQR_50'] = IQR_50
snow_extent_df['days'] = days

In [13]:
month_day = [31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30]
new_list = []

j = 0 
for i in range(0,len(month_day)):
    j+=month_day[i]
    new_list.append(j)
     
print(new_list)

[31, 61, 92, 123, 151, 182, 212, 243, 273, 304, 335, 365]


In [14]:
# Create a list of years to graph. legend rank allows lets you order where the lines are located on the chart. 
for i in range(len(snow_ds)):
    print("""go.Scatter("""
        """name = '""" + str(i + 2001) + """', """
        """y = snow_extent_df['"""+ str(i + 2001) + """'], x = snow_extent_df['days'], """
        """mode = 'lines', legendrank = """ + str(19-i) + """),"""
    )

go.Scatter(name = '2001', y = snow_extent_df['2001'], x = snow_extent_df['days'], mode = 'lines', legendrank = 19),
go.Scatter(name = '2002', y = snow_extent_df['2002'], x = snow_extent_df['days'], mode = 'lines', legendrank = 18),
go.Scatter(name = '2003', y = snow_extent_df['2003'], x = snow_extent_df['days'], mode = 'lines', legendrank = 17),
go.Scatter(name = '2004', y = snow_extent_df['2004'], x = snow_extent_df['days'], mode = 'lines', legendrank = 16),
go.Scatter(name = '2005', y = snow_extent_df['2005'], x = snow_extent_df['days'], mode = 'lines', legendrank = 15),
go.Scatter(name = '2006', y = snow_extent_df['2006'], x = snow_extent_df['days'], mode = 'lines', legendrank = 14),
go.Scatter(name = '2007', y = snow_extent_df['2007'], x = snow_extent_df['days'], mode = 'lines', legendrank = 13),
go.Scatter(name = '2008', y = snow_extent_df['2008'], x = snow_extent_df['days'], mode = 'lines', legendrank = 12),
go.Scatter(name = '2009', y = snow_extent_df['2009'], x = snow_extent_df

In [16]:
#Plot the figure. 
fig = go.Figure([

#create median line
go.Scatter(
    name = 'Median',
    y = snow_extent_df['IQR_50'],
    x = snow_extent_df['days'],
    mode = 'lines',
    legendgroup = 'IQR',
    legendgrouptitle_text="<b>Interquartile Range</b>",
    legendrank = 20,
    line=dict(color='rgb(31, 119, 180)'),
),
#Create IQR 75 line
go.Scatter(
        name = 'IQR 75',
        y = snow_extent_df['IQR_75'],
        x = snow_extent_df['days'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        legendgroup = 'IQR',
        showlegend = False
    ),
    #Create IQR 25 fill 
    go.Scatter(
        name='IQR 25',
        y = snow_extent_df['IQR_25'],
        x = snow_extent_df['days'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        legendgroup = 'IQR',
        showlegend = False
    ),
    go.Scatter(
        name = 'Average Snow Cover',
        y = snow_extent_df['Average Snow Cover'],
        x = snow_extent_df['days'],
        mode = 'lines',
        legendgroup = 'Average',
        legendgrouptitle_text = '<b>Average</b>',
        legendrank = 21
    ),

#Create lines for each respective year
go.Scatter(name = '2001', y = snow_extent_df['2001'], x = snow_extent_df['days'], mode = 'lines', legendrank = 19),
go.Scatter(name = '2002', y = snow_extent_df['2002'], x = snow_extent_df['days'], mode = 'lines', legendrank = 18),
go.Scatter(name = '2003', y = snow_extent_df['2003'], x = snow_extent_df['days'], mode = 'lines', legendrank = 17),
go.Scatter(name = '2004', y = snow_extent_df['2004'], x = snow_extent_df['days'], mode = 'lines', legendrank = 16),
go.Scatter(name = '2005', y = snow_extent_df['2005'], x = snow_extent_df['days'], mode = 'lines', legendrank = 15),
go.Scatter(name = '2006', y = snow_extent_df['2006'], x = snow_extent_df['days'], mode = 'lines', legendrank = 14),
go.Scatter(name = '2007', y = snow_extent_df['2007'], x = snow_extent_df['days'], mode = 'lines', legendrank = 13),
go.Scatter(name = '2008', y = snow_extent_df['2008'], x = snow_extent_df['days'], mode = 'lines', legendrank = 12),
go.Scatter(name = '2009', y = snow_extent_df['2009'], x = snow_extent_df['days'], mode = 'lines', legendrank = 11),
go.Scatter(name = '2010', y = snow_extent_df['2010'], x = snow_extent_df['days'], mode = 'lines', legendrank = 10),
go.Scatter(name = '2011', y = snow_extent_df['2011'], x = snow_extent_df['days'], mode = 'lines', legendrank = 9),
go.Scatter(name = '2012', y = snow_extent_df['2012'], x = snow_extent_df['days'], mode = 'lines', legendrank = 8),
go.Scatter(name = '2013', y = snow_extent_df['2013'], x = snow_extent_df['days'], mode = 'lines', legendrank = 7),
go.Scatter(name = '2014', y = snow_extent_df['2014'], x = snow_extent_df['days'], mode = 'lines', legendrank = 6),
go.Scatter(name = '2015', y = snow_extent_df['2015'], x = snow_extent_df['days'], mode = 'lines', legendrank = 5),
go.Scatter(name = '2016', y = snow_extent_df['2016'], x = snow_extent_df['days'], mode = 'lines', legendrank = 4),
go.Scatter(name = '2017', y = snow_extent_df['2017'], x = snow_extent_df['days'], mode = 'lines', legendrank = 3),
go.Scatter(name = '2018', y = snow_extent_df['2018'], x = snow_extent_df['days'], mode = 'lines', legendrank = 2),
go.Scatter(name = '2019', y = snow_extent_df['2019'], x = snow_extent_df['days'], mode = 'lines', legendrank = 1)

])


# Can change default "off" variables. Right now, the only variable visible is year_2019
variables_to_hide = [
'2001',
'2002',
'2003',
'2004',
'2005',
'2006',
'2007',
'2008',
'2009',
'2010',
'2011',
'2012',
'2013',
'2014',
'2015',
'2016',
'2017',
'2018',
'Average Snow Cover']
fig.for_each_trace(lambda trace: trace.update(visible="legendonly") 
                   if trace.name in variables_to_hide else ())

fig.update_layout(
    title = "<b> Annual Snow Cover Area: Sierra Nevada Region </b> <br> <sup>2001-2019</sup></br>",
    legend_title="<b>Year</b>",
    autosize=False,
    width=1200,
    height=700,
    template = 'none',
    font=dict(
        size=16),
xaxis = dict(
        tickmode = 'array',
        tickvals = [1, 31, 61, 92, 123, 151, 182, 212, 243, 273, 304, 335, 365],
        ticktext = ['<b>October</b>', '<b>November</b>', '<b>December</b>', '<b>January</b>', '<b>February</b>', '<b>March</b>', '<b>April</b>', '<b>May</b>', 
        '<b>June</b>', '<b>July', '<b>August</b>', "<b>September</b>", "<b>October</b>"],
        tickfont = dict(size=12))
)

fig.update_xaxes(title_text = "", gridcolor = 'lightgrey', gridwidth = 0.1)
fig.update_yaxes(title_text = "<b> Area (Thousands of Square Kilometers) </b>", 
    title_font = {"size": 15}, gridcolor = 'lightgrey', gridwidth = 0.1)

fig.show()

In [17]:
# fig.write_html('snow_cover_figure.html')

# Albedo

## Packages 

In [1]:
import glob
import os
from osgeo import gdal
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [2]:
# Glob together all of the Snow Fraction datasets.
albedo = glob.glob('albedo/*.h5')
albedo

['albedo\\SierraAlbedo2001.h5',
 'albedo\\SierraAlbedo2002.h5',
 'albedo\\SierraAlbedo2003.h5',
 'albedo\\SierraAlbedo2004.h5',
 'albedo\\SierraAlbedo2005.h5',
 'albedo\\SierraAlbedo2006.h5',
 'albedo\\SierraAlbedo2007.h5',
 'albedo\\SierraAlbedo2008.h5',
 'albedo\\SierraAlbedo2009.h5',
 'albedo\\SierraAlbedo2010.h5',
 'albedo\\SierraAlbedo2011.h5',
 'albedo\\SierraAlbedo2012.h5',
 'albedo\\SierraAlbedo2013.h5',
 'albedo\\SierraAlbedo2014.h5',
 'albedo\\SierraAlbedo2015.h5',
 'albedo\\SierraAlbedo2016.h5',
 'albedo\\SierraAlbedo2017.h5',
 'albedo\\SierraAlbedo2018.h5',
 'albedo\\SierraAlbedo2019.h5']

In [14]:
# Get subdatasets of first snow fraction dataset ('SierraAlbedo2001.h5').
dataset = gdal.Open(albedo[0], gdal.GA_ReadOnly)


#Changes the selected dataset into an array.

albedo_array = dataset.ReadAsArray()

albedo_float = albedo_array.astype('float')
albedo_float[albedo_float == 65535] = np.nan

albedo_year_2001_day_20 = albedo_float[20:,:]/10000

albedo_mean_20  = np.nanmean(albedo_year_2001_day_20)

AttributeError: 'NoneType' object has no attribute 'astype'

In [7]:
# # Create an empty list to populate yearly for loop values with
# mean_albedo = []
# for i in range(len(albedo)):
#     dataset = gdal.Open(albedo[i], gdal.GA_ReadOnly)


#     #Changes the selected dataset into an array.
#     albedo_array = dataset.ReadAsArray()

#     albedo_float = albedo_array.astype('float')
#     albedo_float[albedo_float == 65535] = np.nan

#     albedo_divisor = albedo_float/10000
#     #Create an empty list to populate daily mean values per year with
#     new_list = []
#     #Takes the sum of the average snow fraction area per cell per given day in a year
#     for j in range(len(albedo_float)):
        
#         albedo_day = albedo_divisor[j:,:]
#         albedo_day_mean = np.nanmean(albedo_day) 
#         #Appends daily values to list
#         new_list.append(albedo_day_mean)
#     #Appends yearly values to list. 
#     mean_albedo.append(new_list)

KeyboardInterrupt: 

In [ ]:
# # Create empty lists for row names and col names
# row_names = []
# col_names = []
# # Create names for each day
# for i in range(366) :
#     col_names.append('Day ' + str(i + 1))
# # Create names for year 
# for j in range(len(albedo)):
#     row_names.append(str(j + 2001))
# #Create dataframe and append with names. 
# albedo_df = pd.DataFrame(mean_albedo, index = row_names)
# #Transpose dataframe so years are columns and days are rows.
# albedo_df = pd.DataFrame.transpose(albedo_df)

In [ ]:
#albedo_df.to_csv('albedo_rates.csv')

In [ ]:
albedo_df = pd.read_csv(r'albedo_df.csv')

In [ ]:
#Now need to convert NA Values for non-leap years
albedo_df = albedo_df.fillna(0)
albedo_df
# Next, need to create a single column of mean values. 
albedo_df['Average Albedo'] = albedo_df.mean(axis = 1)

In [ ]:
# Create empty list to input with for loop
IQR_25 = []
IQR_50 = []
IQR_75 = []
days = []
for i in range(len(albedo_df)): 
    #Takes the IQR of each day (25/75)
    Q1 = np.percentile(albedo_df.iloc[i], 25)
    Q2 = np.percentile(albedo_df.iloc[i], 50)
    Q3 = np.percentile(albedo_df.iloc[i], 75)
    #appends list with IQR outputs
    IQR_25.append(Q1)
    IQR_50.append(Q2)
    IQR_75.append(Q3)
    #Creates day list to append dataset with
    days.append(i + 1)

In [ ]:
#Appends list for loop lists
albedo_df['IQR_25'] = IQR_25
albedo_df['IQR_75'] = IQR_75
albedo_df['IQR_50'] = IQR_50
albedo_df['days'] = days

In [ ]:
month_day = [31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30]
new_list = []

j = 0 
for i in range(0,len(month_day)):
    j+=month_day[i]
    new_list.append(j)
     
print(new_list)

In [ ]:
# Create a list of years to graph. legend rank allows lets you order where the lines are located on the chart. 
for i in range(len(albedo)):
    print("""go.Scatter("""
        """name = '""" + str(i + 2001) + """', """
        """y = albedo_df['"""+ str(i + 2001) + """'], x = albedo_df['days'], """
        """mode = 'lines', legendrank = """ + str(19-i) + """),"""
    )

In [ ]:
#Plot the figure. 
fig2 = go.Figure([

#create median line
go.Scatter(
    name = 'Median',
    y = albedo_df['IQR_50'],
    x = albedo_df['days'],
    mode = 'lines',
    legendgroup = 'IQR',
    legendgrouptitle_text="<b>Interquartile Range</b>",
    legendrank = 20,
    line=dict(color='rgb(31, 119, 180)'),
),
#Create IQR 75 line
go.Scatter(
        name = 'IQR 75',
        y = albedo_df['IQR_75'],
        x = albedo_df['days'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        legendgroup = 'IQR',
        showlegend = False
    ),
    #Create IQR 25 fill 
    go.Scatter(
        name='IQR 25',
        y = albedo_df['IQR_25'],
        x = albedo_df['days'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        legendgroup = 'IQR',
        showlegend = False
    ),
    go.Scatter(
        name = 'Average Albedo',
        y = albedo_df['Average Albedo'],
        x = albedo_df['days'],
        mode = 'lines',
        legendgroup = 'Average',
        legendgrouptitle_text = '<b>Average</b>',
        legendrank = 21
    ),

#Create lines for each respective line 
go.Scatter(name = '2001', y = albedo_df['2001'], x = albedo_df['days'], mode = 'lines', legendrank = 19),
go.Scatter(name = '2002', y = albedo_df['2002'], x = albedo_df['days'], mode = 'lines', legendrank = 18),
go.Scatter(name = '2003', y = albedo_df['2003'], x = albedo_df['days'], mode = 'lines', legendrank = 17),
go.Scatter(name = '2004', y = albedo_df['2004'], x = albedo_df['days'], mode = 'lines', legendrank = 16),
go.Scatter(name = '2005', y = albedo_df['2005'], x = albedo_df['days'], mode = 'lines', legendrank = 15),
go.Scatter(name = '2006', y = albedo_df['2006'], x = albedo_df['days'], mode = 'lines', legendrank = 14),
go.Scatter(name = '2007', y = albedo_df['2007'], x = albedo_df['days'], mode = 'lines', legendrank = 13),
go.Scatter(name = '2008', y = albedo_df['2008'], x = albedo_df['days'], mode = 'lines', legendrank = 12),
go.Scatter(name = '2009', y = albedo_df['2009'], x = albedo_df['days'], mode = 'lines', legendrank = 11),
go.Scatter(name = '2010', y = albedo_df['2010'], x = albedo_df['days'], mode = 'lines', legendrank = 10),
go.Scatter(name = '2011', y = albedo_df['2011'], x = albedo_df['days'], mode = 'lines', legendrank = 9),
go.Scatter(name = '2012', y = albedo_df['2012'], x = albedo_df['days'], mode = 'lines', legendrank = 8),
go.Scatter(name = '2013', y = albedo_df['2013'], x = albedo_df['days'], mode = 'lines', legendrank = 7),
go.Scatter(name = '2014', y = albedo_df['2014'], x = albedo_df['days'], mode = 'lines', legendrank = 6),
go.Scatter(name = '2015', y = albedo_df['2015'], x = albedo_df['days'], mode = 'lines', legendrank = 5),
go.Scatter(name = '2016', y = albedo_df['2016'], x = albedo_df['days'], mode = 'lines', legendrank = 4),
go.Scatter(name = '2017', y = albedo_df['2017'], x = albedo_df['days'], mode = 'lines', legendrank = 3),
go.Scatter(name = '2018', y = albedo_df['2018'], x = albedo_df['days'], mode = 'lines', legendrank = 2),
go.Scatter(name = '2019', y = albedo_df['2019'], x = albedo_df['days'], mode = 'lines', legendrank = 1)

])

#fig2.show()
#data=fig1.data + fig2.data
#fig3 = go.Figure(data)

# Can change default "off" variables. Right now, the only variable visible is year_2001
variables_to_hide = [
'2001',
'2002',
'2003',
'2004',
'2005',
'2006',
'2007',
'2008',
'2009',
'2010',
'2011',
'2012',
'2013',
'2014',
'2015',
'2016',
'2017',
'2018',
'Average Albedo']
fig2.for_each_trace(lambda trace: trace.update(visible="legendonly") 
                   if trace.name in variables_to_hide else ())

fig2.update_layout(
    title = "<b> Annual Snow Albedo Percentages: Sierra Nevada Region </b> <br> <sup>2001-2019</sup></br>",
    legend_title="<b>Year</b>",
    autosize=False,
    width=1200,
    height=700,
    template = 'none',
    font=dict(
        size=16),
    yaxis_range =[0.56,.8],
    xaxis_range = [1, 365],
xaxis = dict(
        tickmode = 'array',
        tickvals = [1, 31, 61, 92, 123, 151, 182, 212, 243, 273, 304, 335, 365],
        ticktext = ['<b>October</b>', '<b>November</b>', '<b>December</b>', '<b>January</b>', '<b>February</b>', '<b>March</b>', '<b>April</b>', '<b>May</b>', 
        '<b>June</b>', '<b>July', '<b>August</b>', "<b>September</b>", "<b>October</b>"],
        tickfont = dict(size=12))
)

fig2.update_xaxes(title_text = "", gridcolor = 'lightgrey', gridwidth = 0.1)
fig2.update_yaxes(title_text = "<b> Percentage </b>", 
    title_font = {"size": 15}, gridcolor = 'lightgrey', gridwidth = 0.1)

fig2.show()